Data Loading

In [2]:
import pandas as pd



In [3]:
df = pd.read_csv('dataset/synthetic_logs.csv')
df


,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert


In [4]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [5]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

Installation of Requirements

In [6]:
!pip install scikit-learn
from sklearn.cluster import DBSCAN


In [7]:
!pip install -U sentence-transformers


Clustering

In [8]:
from sentence_transformers import SentenceTransformer


c:\Users\antar\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['log_message'].tolist())
# embeddings[:2]
embeddings_df = pd.DataFrame(embeddings)
embeddings_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.102940,0.033546,-0.022026,0.001551,-0.009869,-0.178956,-0.063441,-0.060176,0.028111,0.059962,...,-0.074033,0.018422,-0.008649,-0.088356,-0.059198,-0.000101,0.112914,0.004578,-0.042597,0.003226
1,0.008046,-0.035739,0.049387,-0.007192,0.000618,-0.090568,0.057560,-0.043945,0.017475,0.052120,...,0.014534,-0.046097,0.050255,-0.072703,0.029278,0.056266,-0.038636,0.015383,-0.062310,-0.027747


In [10]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.1, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings_df)

df['cluster'] = clusters
df.head(10)

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert,0
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert,3
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert,5
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert,0


In [11]:
df[df.cluster == 1].head(10)

,timestamp,source,log_message,target_label,complexity,cluster
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,bert,1
1901,9/22/2025 2:52,ThirdPartyAPI,Email service experienced a mail sending issue,Error,bert,1


Sorting the Clusters

In [13]:
custer_counts = df.cluster.value_counts()
large_clusters = custer_counts[custer_counts > 10].index

for cluster in large_clusters:
    print(f'Cluster {cluster}:')
    print(df[df.cluster == cluster]['log_message'].head(10).to_string(index=False))
    print('')

Cluster 0:
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...
nova.osapi_compute.wsgi.server [req-d4f8d0c2-4f...
nova.osapi_compute.wsgi.server [req-6fe0e366-f2...
nova.osapi_compute.wsgi.server [req-5f1c2027-e1...
nova.osapi_compute.wsgi.server [req-945d1f31-a2...
nova.osapi_compute.wsgi.server [req-75bc6269-85...

Cluster 16:
nova.metadata.wsgi.server [-] 10.11.21.138,10.1...
nova.metadata.wsgi.server [req-61196723-e034-48...
nova.metadata.wsgi.server [req-7d3eeb2d-3948-43...
nova.metadata.wsgi.server [-] 10.11.21.137,10.1...
nova.metadata.wsgi.server [req-e0d4ce94-e0cb-41...
nova.metadata.wsgi.server [req-c1795e2c-0a17-4a...
nova.metadata.wsgi.server [-] 10.11.21.139,10.1...
nova.metadata.wsgi.server [-] 10.11.21.136,10.1...
nova.metadata.wsgi.server [req-e5e061d6-f2aa-46...
nova.me

Regex Classification

In [16]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None

In [17]:
classify_with_regex('User User123 logged in.')

'User Action'

In [18]:
classify_with_regex('Account with ID 123 created by admin.')

'User Action'

In [19]:
classify_with_regex('hey, i got you!')

In [21]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)    
df[df['regex_label'].notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,9,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,10,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,13,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,27,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,45,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,45,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,15,System Notification


In [22]:
df[df['regex_label'].isna()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,516,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,16,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,517,None


In [ ]:
df[df['regex_label'].notnull()].shape

(500, 7)

In [25]:
df[df['regex_label'].isna()].shape

(1910, 7)

In [27]:
df_non_regex = df[df['regex_label'].isna()]
df_non_regex.shape

(1910, 7)

In [28]:
df_non_legacy = df_non_regex[df_non_regex.source != 'LegacyCRM']
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [30]:
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())
# filtered_embeddings[:2]
filtered_embeddings= pd.DataFrame(filtered_embeddings)
filtered_embeddings.head(2)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.102940,0.033546,-0.022026,0.001551,-0.009869,-0.178956,-0.063441,-0.060176,0.028111,0.059962,...,-0.074033,0.018422,-0.008649,-0.088356,-0.059198,-0.000101,0.112914,0.004578,-0.042597,0.003226
1,0.008046,-0.035739,0.049387,-0.007192,0.000618,-0.090568,0.057560,-0.043945,0.017475,0.052120,...,0.014534,-0.046097,0.050255,-0.072703,0.029278,0.056266,-0.038636,0.015383,-0.062310,-0.027747


In [32]:
X = filtered_embeddings
y = df_non_legacy['target_label'].values

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)


                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [37]:
import joblib
joblib.dump(clf,'../models/log_classifier.joblib')

['../models/log_classifier.joblib']